<a href="https://colab.research.google.com/github/sushankhadka12/AI-Lab3-sushankhadka021390/blob/main/Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
from collections import deque
import heapq

class VacuumEnvironment:
    def __init__(self, size=10, dirt_prob=0.2):
        self.size = size
        self.grid = [[random.random() < dirt_prob for _ in range(size)] for _ in range(size)]
        self.agent_pos = (random.randint(0, size-1), random.randint(0, size-1))

    def is_dirty(self, pos=None):
        if pos is None:
            pos = self.agent_pos
        x, y = pos
        return self.grid[x][y]

    def clean_cell(self, pos=None):
        if pos is None:
            pos = self.agent_pos
        x, y = pos
        self.grid[x][y] = False

    def move_agent(self, direction):
        x, y = self.agent_pos
        if direction == 'up' and x > 0:
            self.agent_pos = (x-1, y)
        elif direction == 'down' and x < self.size-1:
            self.agent_pos = (x+1, y)
        elif direction == 'left' and y > 0:
            self.agent_pos = (x, y-1)
        elif direction == 'right' and y < self.size-1:
            self.agent_pos = (x, y+1)
        return self.agent_pos

    def display(self):
        for i in range(self.size):
            row = []
            for j in range(self.size):
                cell = 'A' if (i,j) == self.agent_pos else 'D' if self.grid[i][j] else 'C'
                row.append(cell)
            print(' '.join(row))
        print()

    def all_clean(self):
        return not any(any(row) for row in self.grid)

In [ ]:
def simple_reflex_agent(env, steps=20):
    print("Simple Reflex Agent Simulation")
    print("Initial Grid:")
    env.display()

    for step in range(1, steps+1):
        print(f"Step {step}:")
        if env.is_dirty():
            action = 'suck'
            env.clean_cell()
        else:
            possible_actions = ['up', 'down', 'left', 'right']
            valid_actions = []
            for action in possible_actions:
                x, y = env.agent_pos
                if (action == 'up' and x > 0) or \
                   (action == 'down' and x < env.size-1) or \
                   (action == 'left' and y > 0) or \
                   (action == 'right' and y < env.size-1):
                    valid_actions.append(action)

            if valid_actions:
                action = random.choice(valid_actions)
                env.move_agent(action)
            else:
                action = 'noop'

        print(f"Action: {action}")
        env.display()

In [ ]:
def goal_based_agent(env):
    print("Goal-Based Agent Simulation")
    print("Initial Grid:")
    env.display()

    dirty_cells = set()
    for i in range(env.size):
        for j in range(env.size):
            if env.grid[i][j]:
                dirty_cells.add((i, j))

    actions = []

    while dirty_cells:
        queue = deque()
        queue.append((env.agent_pos, []))
        visited = set()
        visited.add(env.agent_pos)
        path = []

        while queue:
            pos, current_path = queue.popleft()

            if pos in dirty_cells:
                path = current_path
                target = pos
                break

            x, y = pos
            neighbors = []
            if x > 0: neighbors.append((x-1, y, 'up'))
            if x < env.size-1: neighbors.append((x+1, y, 'down'))
            if y > 0: neighbors.append((x, y-1, 'left'))
            if y < env.size-1: neighbors.append((x, y+1, 'right'))

            for nx, ny, action in neighbors:
                if (nx, ny) not in visited:
                    visited.add((nx, ny))
                    queue.append(((nx, ny), current_path + [action]))

        for action in path:
            env.move_agent(action)
            actions.append(action)
            print(f"Move: {action}")
            env.display()

        if env.is_dirty(target):
            env.agent_pos = target
            env.clean_cell()
            actions.append('suck')
            dirty_cells.remove(target)
            print("Action: suck")
            env.display()

    print("All cells clean!")
    print("Sequence of actions:", actions)
    print("Final Grid:")
    env.display()

In [ ]:
def utility_based_agent(env):
    print("Utility-Based Agent Simulation")
    print("Initial Grid:")
    env.display()


    dirty_cells = set()
    for i in range(env.size):
        for j in range(env.size):
            if env.grid[i][j]:
                dirty_cells.add((i, j))

    actions = []
    total_utility = 0

    while dirty_cells:
        best_action = None
        best_utility = -float('inf')


        possible_actions = []
        x, y = env.agent_pos


        if env.is_dirty():
            possible_actions.append(('suck', None))


        if x > 0: possible_actions.append(('move', 'up'))
        if x < env.size-1: possible_actions.append(('move', 'down'))
        if y > 0: possible_actions.append(('move', 'left'))
        if y < env.size-1: possible_actions.append(('move', 'right'))

        for action_type, direction in possible_actions:
            utility = 0

            if action_type == 'suck':
                utility = 5  # +5 for cleaning
            else:
                # Calculate utility for move actions
                new_pos = None
                if direction == 'up': new_pos = (x-1, y)
                elif direction == 'down': new_pos = (x+1, y)
                elif direction == 'left': new_pos = (x, y-1)
                elif direction == 'right': new_pos = (x, y+1)


                if new_pos in dirty_cells:
                    utility = 4
                else:

                    min_distance = min(abs(nx-new_pos[0]) + abs(ny-new_pos[1])
                                    for (nx, ny) in dirty_cells)
                    utility = -min_distance

                utility -= 1

            if utility > best_utility:
                best_utility = utility
                best_action = (action_type, direction)

        action_type, direction = best_action
        if action_type == 'suck':
            env.clean_cell()
            dirty_cells.discard(env.agent_pos)
            actions.append('suck')
            total_utility += 5
            print("Action: suck")
        else:
            env.move_agent(direction)
            actions.append(f'move {direction}')
            total_utility -= 1
            print(f"Action: move {direction}")

        env.display()

    print("All cells clean!")
    print("Sequence of actions:", actions)
    print("Total utility:", total_utility)
    print("Final Grid:")
    env.display()

In [ ]:
def simulate_agents():
    print("=== SIMPLE REFLEX AGENT ===")
    env1 = VacuumEnvironment(size=4, dirt_prob=0.3)
    simple_reflex_agent(env1, steps=20)

    print("\n=== GOAL-BASED AGENT ===")
    env2 = VacuumEnvironment(size=4, dirt_prob=0.3)
    goal_based_agent(env2)

    print("\n=== UTILITY-BASED AGENT ===")
    env3 = VacuumEnvironment(size=4, dirt_prob=0.3)
    utility_based_agent(env3)

simulate_agents()

=== SIMPLE REFLEX AGENT ===
Simple Reflex Agent Simulation
Initial Grid:
D C C C
A D C C
C D C C
C C C C

Step 1:
Action: down
D C C C
C D C C
A D C C
C C C C

Step 2:
Action: down
D C C C
C D C C
C D C C
A C C C

Step 3:
Action: right
D C C C
C D C C
C D C C
C A C C

Step 4:
Action: left
D C C C
C D C C
C D C C
A C C C

Step 5:
Action: up
D C C C
C D C C
A D C C
C C C C

Step 6:
Action: right
D C C C
C D C C
C A C C
C C C C

Step 7:
Action: suck
D C C C
C D C C
C A C C
C C C C

Step 8:
Action: right
D C C C
C D C C
C C A C
C C C C

Step 9:
Action: left
D C C C
C D C C
C A C C
C C C C

Step 10:
Action: up
D C C C
C A C C
C C C C
C C C C

Step 11:
Action: suck
D C C C
C A C C
C C C C
C C C C

Step 12:
Action: left
D C C C
A C C C
C C C C
C C C C

Step 13:
Action: down
D C C C
C C C C
A C C C
C C C C

Step 14:
Action: down
D C C C
C C C C
C C C C
A C C C

Step 15:
Action: right
D C C C
C C C C
C C C C
C A C C

Step 16:
Action: left
D C C C
C C C C
C C C C
A C C C

Step 17:
Action: up
D C